# Setup

In [1]:
from __future__ import annotations
import contextlib
import json
import math
import re
from functools import reduce
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Tuple

import matplotlib as mpl
import matplotlib.ticker as plticker
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

## Config

In [2]:
INSTANCE_DIR = Path('./instances')
TIMEOUT_SECS = 10
MIN_HARD_SECS = 1

WIDTH_1COL = 3.335
WIDTH_2COL = 6.808

SCATTER_SETTINGS = {
    's': 20,
    'alpha': 0.7,
    'linewidths': 0,
}
BOXPLOT_SETTINGS = {
    # Fill boxes in white so that background lines don't appear to go
    # through the boxes
    'patch_artist': True,
    'boxprops': {
        'facecolor': 'white',
    }
}
BACKGROUND_LINE_SETTINGS = {
    'color': 'gray',
    'linewidth': 0.75,
    'zorder': -1,
}

## Matplotlib config

In [3]:
mpl.rc('font', family='serif', serif='Computer Modern')
mpl.rc('text', usetex=True)
plt.rcParams.update({
    'text.latex.preamble': r'''
        \usepackage{amsmath}
    '''
})

## Pre-processing results

In [5]:
result_dirs = [d for d in Path('results').glob('*') if d.is_dir()]
for results_dir in tqdm(result_dirs):
    data = [json.loads(f.read_text()) for f in results_dir.glob('**/*.json')]
    with Path(f'results/{results_dir.name}.json').open('w') as f:
        json.dump(data, f)

  0%|          | 0/21 [00:00<?, ?it/s]

## Loading results

In [4]:
all_instance_names = [f.name for f in INSTANCE_DIR.glob('*.dat')]


def fix_column_names(col):
    if col.startswith('__'):
        return col[2:]
    return col


def load_combined_json(path: Path) -> pd.DataFrame:
    with path.open() as f:
        data = json.load(f)
    df = pd.json_normalize(data, sep='__').rename(columns=fix_column_names)
    df.set_index('file_name', inplace=True)
    return df.reindex(all_instance_names)



json_files = list(Path('results').glob('*.json'))
dfs_by_experiment = {f.stem: load_combined_json(f) for f in tqdm(json_files)}

  0%|          | 0/21 [00:00<?, ?it/s]

## Loading Gurobi results

In [5]:
# Gurobi log parsing
GUROBI_REGEXES = [
    re.compile(regex, re.MULTILINE) for regex in (
        r'^Presolve time: (.*)s$',
        r'^Root relaxation: .*?, .*? iterations, (.*?) seconds$',
        r'^Explored .*? nodes \(.*? simplex iterations\) in (.*?) seconds$',
    )
]


def gurobi_parse_time(log_file: Path) -> Optional[float]:
    text = Path(log_file).read_text()
    matches = [regex.search(text) for regex in GUROBI_REGEXES]
    # Solving finished if the last message matched by the last regex appears
    if matches[-1] is None:
        return None
    return sum(float(m[1]) for m in matches if m is not None)


gurobi_logs = list(Path('gurobi-logs').glob('*.log'))
gurobi_runtimes = {(f.stem + '.dat'): gurobi_parse_time(f)
                   for f in tqdm(gurobi_logs)}


  0%|          | 0/4256 [00:00<?, ?it/s]

## Getting instance sizes

In [6]:
def get_instance_size(p: Path) -> int:
    with p.open() as f:
        # Skip initial line containing node and edge count
        next(f)
        return sum(int(line.split(maxsplit=1)[0]) for line in f)


instances = list(INSTANCE_DIR.glob('*.dat'))
instance_size = {f.name: get_instance_size(f) for f in tqdm(instances)}

  0%|          | 0/4256 [00:00<?, ?it/s]

## Classifying instances

In [7]:
# Hard instance for solver: required >= MIN_HARD_SECS seconds to finish
# Easy: not hard

default_df = dfs_by_experiment['default']
easy_instances_findminhs = set(
    default_df[default_df.runtimes__total < MIN_HARD_SECS].index)
easy_instances_gurobi = {
    name
    for name, runtime in gurobi_runtimes.items()
    if runtime is not None and runtime < MIN_HARD_SECS
}
easy_instances_both = easy_instances_findminhs & easy_instances_gurobi
hard_instances_findminhs = set(all_instance_names) - easy_instances_findminhs

num_hard_finished_findminhs = len(
    default_df[default_df.index.isin(hard_instances_findminhs)].dropna())
num_hard_finished_gurobi = sum(1 for t in gurobi_runtimes.values()
                               if t is not None and t >= MIN_HARD_SECS)
unfinished_gurobi = {
    name
    for name, runtime in gurobi_runtimes.items() if runtime is None
}
unfinished_findminhs = set(default_df[default_df.runtimes__total.isna()].index)

print(f'Hard for findminhs: {len(hard_instances_findminhs)}')
print(f'Hard & finished findminhs: {num_hard_finished_findminhs}')
print('Hard for gurobi: '
      f'{len(all_instance_names) - len(easy_instances_gurobi)}')
print(f'Hard & finished gurobi: {num_hard_finished_gurobi}')
print()
print('Hard only for findminhs: '
      f'{len(easy_instances_gurobi - easy_instances_findminhs)}')
print('Hard only for gurobi: '
      f'{len(easy_instances_findminhs - easy_instances_gurobi)}')
print()
print('Finished only for findminhs: '
      f'{len(unfinished_gurobi - unfinished_findminhs)}')
print('Finished only for gurobi: '
      f'{len(unfinished_findminhs - unfinished_gurobi)}')


def get_df_hard_finished(name: str) -> pd.DataFrame:
    """
    Returns a data frame for an experiment, containing only the instances
    that were hard for the default setting and finished in this experiment.
    """
    df = dfs_by_experiment[name]
    return df[df.index.isin(hard_instances_findminhs)].dropna()


def get_dfs_hard_finished(names: Iterable[str]) -> Dict[pd.DataFrame]:
    """
    Returns data frames for multiple experiments, containing only the
    instances that were hard for the default setting and finished in
    all of the given experiments.

    A message is printed listing how many instances were removed since
    they didn't finish in all of the given experiments.
    """
    dfs = {name: get_df_hard_finished(name) for name in names}
    indices = [df.index for df in dfs.values()]
    index_intersection = reduce(lambda i1, i2: i1.intersection(i2), indices)
    index_union = reduce(lambda i1, i2: i1.union(i2), indices)
    dropped = set(index_union) - set(index_intersection)
    print(f'Dropping {len(dropped)} instances since they did not finish in '
          'all experiments')
    print(f'Dropped instances: {dropped}')
    return {
        name: df[df.index.isin(index_intersection)].copy()
        for name, df in dfs.items()
    }


Hard for findminhs: 142
Hard & finished findminhs: 48
Hard for gurobi: 319
Hard & finished gurobi: 290

Hard only for findminhs: 0
Hard only for gurobi: 177

Finished only for findminhs: 0
Finished only for gurobi: 65


## Utilities

In [9]:
@contextlib.contextmanager
def make_plot(
        name: str,
        figsize: Tuple[int, int]) -> Iterable[Tuple[plt.Figure, plt.Axes]]:
    fig = plt.figure(figsize=figsize)
    ax = fig.subplots()
    yield fig, ax
    fig.tight_layout()
    fig.savefig(f'plots/{name}.pdf')
    plt.close('all')


def root_lower_bounds(df: pd.DataFrame) -> pd.Series:
    lower_bound_names = [
        'max_degree', 'sum_degree', 'efficiency', 'packing', 'sum_over_packing'
    ]
    return reduce(np.maximum,
                  (df[f'root_bounds__{name}'] for name in lower_bound_names))


# Runtime comparison with Gurobi

In [55]:
df = dfs_by_experiment['default'].copy()
df['runtimes__gurobi'] = df.index.map(gurobi_runtimes)

# Remove all instances that don't finish for either
df = df[df.runtimes__gurobi.notna() & df.runtimes__total.notna()]

# Gurobis log entries only have two digits of precision.
# Therefore (and since the low values are not that interesting anyways)
# clip all values below 0.01 to 0.01.
x = df.runtimes__gurobi.clip(lower=0.01)
y = df.runtimes__total.clip(lower=0.01)

# Round up/down to nearest power 10 plus margin
mx_log = math.log10(max(x.max(), y.max()))
mx = 10**max(math.ceil(mx_log), mx_log + 0.1)
mn_log = math.log10(min(x.min(), y.min()))
mn = 10**min(math.floor(mn_log), mn_log - 0.1)

with make_plot('runtime-vs-gurobi', (WIDTH_1COL, WIDTH_1COL)) as (fig, ax):
    ax.scatter(x, y, **SCATTER_SETTINGS)

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_aspect('equal')

    mn = 0.01
    mx = max(ax.get_xlim()[1], ax.get_ylim()[1])
    ax.set_xlim((mn, mx))
    ax.set_ylim((mn, mx))

    ax.yaxis.set_major_locator(ax.xaxis.get_major_locator())
    ax.yaxis.set_minor_locator(ax.xaxis.get_minor_locator())

    ax.autoscale(enable=False)
    ax.plot((mn, mx), (mn, mx), **BACKGROUND_LINE_SETTINGS)
    for i in range(1, 100):
        if mn * 10**i > mx:
            break
        ax.plot((mn * 10**i, mx), (mn, mx / 10**i),
                **BACKGROUND_LINE_SETTINGS,
                linestyle='dashed')
        ax.plot((mn, mx / 10**i), (mn * 10**i, mx),
                **BACKGROUND_LINE_SETTINGS,
                linestyle='dashed')

    ax.set_xlabel('Runtime Gurobi (s)')
    ax.set_ylabel('Runtime (s)')

# Search space

In [56]:
default_df = get_df_hard_finished('default')


def search_space_plot(name: str,
                      x,
                      xlabel: str,
                      xlog: bool = False) -> None:
    with make_plot(name, (WIDTH_1COL, WIDTH_1COL)) as (fig, ax):
        y = default_df.branching_steps.dropna()
        ax.scatter(x, default_df.branching_steps, **SCATTER_SETTINGS)
        ax.set_yscale('symlog')
        if xlog:
            ax.set_xscale('log')

        ax.set_xlabel(xlabel)
        ax.set_ylabel(r'Search space (\#branching steps)')


lower = root_lower_bounds(default_df)
gap = default_df.root_bounds__greedy_upper - lower
search_space_plot('search-space-bound-gap', gap,
                  r'Bound gap ($\text{upper} - \text{lower}$)')

sizes = default_df.index.map(instance_size)
search_space_plot('search-space-instance-size',
                  sizes,
                  r'$\lVert \mathcal{F} \rVert$',
                  xlog=True)

# Lower bound vs. opt vs. upper bound (for full instance)

In [57]:
default_df = get_df_hard_finished('default')

lower = root_lower_bounds(default_df);
opt = default_df.opt
upper = default_df.root_bounds__greedy_upper

x = lower / opt
y = upper / opt
mx = round(1.1 * y.max(), ndigits = 1)
x_mn = math.floor(10 * 0.95 * x.min()) / 10

with make_plot('bound-gaps', (WIDTH_1COL, WIDTH_1COL)) as (fig, ax):
    ax.scatter(x, y, **SCATTER_SETTINGS)

    ax.set_xlim((x_mn, 1.03))
    ax.set_ylim((0.97, mx))
    ax.xaxis.set_major_locator(plticker.MultipleLocator(base=0.2))
    ax.yaxis.set_major_locator(plticker.MultipleLocator(base=0.5))

    ax.set_xlabel('Lower bound (relative to opt)')
    ax.set_ylabel('Upper bound (relative to opt)')

# Runtime comparison of different operations

In [70]:
parts = {
    'greedy': 'greedy',
    'max_degree_bound': 'max\ndeg.\nbound',
    'efficiency_bound': 'eff.\nbound',
    'packing_bound': 'packing\nbound',
    'sum_over_packing_bound': 'sum\nover\npacking\nbound',
    'forced_vertex': 'forced\nvertex',
    'costly_discard_packing_update': 'costly\ndiscard\npacking\nupdate',
    'costly_discard_packing_from_scratch': 'costly\ndiscard\nrepack',
    'vertex_domination': 'vertex\ndom.',
    'edge_domination': 'edge\ndom.',
    'other': 'other',
}
df = get_df_hard_finished('default')

non_other_runtimes = sum(
    df[col] for col in default_df.columns
    if col.startswith('runtimes__') and col not in (
        'runtimes__total', 'runtimes__applying_reductions'))
runtime_other = df.runtimes__total - non_other_runtimes

data = [(runtime_other if col == 'other' else df[f'runtimes__{col}']) /
        df.runtimes__total for col in parts.keys()]

with make_plot('operations', (WIDTH_2COL, 2.5)) as (fig, ax):
    ax.boxplot(data, **BOXPLOT_SETTINGS)

    font_settings = dict(fontsize='x-small')
    ticklabels = [fr'{num}\%' for num in [0, 20, 40, 60, 80, 100]]
    ax.set_xticklabels(parts.values(), fontdict=font_settings)
    ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels(ticklabels, fontdict=font_settings)

# Comparison of search space with different lower bounds

In [10]:
bounds = {
    'Max degree': 'max-degree-only',
    'Sum degree': 'sum-degree-only',
    'Efficiency': 'efficiency-only',
    'Packing': 'packing-only',
    'Packing\n(local search)': 'packing-local-search-only',
    'Sum\nover packing': 'sum-over-packing-only',
    'Sum\nover packing\n(local search)': 'packing-local-search-only',
}
dfs = get_dfs_hard_finished(list(bounds.values()) + ['default'])
default_df = dfs['default']

idx_nonzero = default_df[default_df.branching_steps > 0].index
num_dropped = len(default_df) - len(idx_nonzero)
print(f'Dropping {num_dropped} instances since they didnt branch when '
      'using default settings')
dfs = {name: df[df.index.isin(idx_nonzero)] for name, df in dfs.items()}
default_df = dfs['default']
print(f'Remaining: {len(default_df)} instances')

data = [
    dfs[name].branching_steps / default_df.branching_steps
    for name in bounds.values()
]

with make_plot('search-space-by-bound', (WIDTH_2COL, 3)) as (fig, ax):
    ax.boxplot(data, **BOXPLOT_SETTINGS)
    ax.set_yscale('log')
    ax.set_xticklabels(bounds.keys())
    ax.set_ylabel('Relative search space')

    ax.autoscale(enable=False)
    ax.plot(ax.get_xlim(), (1, 1), **BACKGROUND_LINE_SETTINGS, linestyle='dashed')

Dropping 1 instances since they did not finish in all experiments
Dropped instances: {'isolet_r7798_c618_r7798_c200_diff_sets.hg.dat'}
Dropping 10 instances since they didnt branch when using default settings
Remaining: 37 instances


# How often are reductions applied? How often are they successfull?

## Bounds

In [72]:
cols = [
    'reductions__max_degree_bound_breaks',
    'reductions__efficiency_degree_bound_breaks',
    'reductions__packing_bound_breaks',
    'reductions__sum_over_packing_bound_breaks',
]
default_df = get_df_hard_finished('default')
total = sum(default_df[col] for col in cols)

idx_nonzero = total[total != 0].index
print(f'Discarding {len(total) - len(idx_nonzero)} instances which didnt have any bound breaks')
default_df = default_df.reindex(idx_nonzero)
total = total.reindex(idx_nonzero)

data = [default_df[col] / total for col in cols]
labels = ['Max\ndegree', 'Efficiency', 'Packing', 'Sum over\npacking']

with make_plot('bound-comparison', (WIDTH_1COL, 2)) as (fig, ax):
    ax.boxplot(data, **BOXPLOT_SETTINGS)
    ax.set_xticklabels(labels)
    ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    ticklabels = [fr'{num}\%' for num in [0, 20, 40, 60, 80, 100]]
    ax.set_yticklabels(ticklabels)

Discarding 1 instances which didnt have any bound breaks


# How many vertices to check in the costly discard from scratch packing rule?

In [69]:
names = [f'from-scratch-{i}' for i in [0] + list(range(1, 20, 2)) if i != 3]
names.append('default')
dfs = get_dfs_hard_finished(names)

base_df = dfs['from-scratch-0']
num_dfs = {
    num: dfs['default' if num == 3 else f'from-scratch-{num}']
    for num in range(1, 20, 2)
}

data = [
    df.runtimes__total / base_df.runtimes__total for df in num_dfs.values()
]

with make_plot('from-scratch', (WIDTH_2COL, 2.5)) as (fix, ax):
    ax.boxplot(data, **BOXPLOT_SETTINGS)

    ax.set_xlabel(r'\#Nodes checked')
    ax.set_ylabel('Runtime (rel. to w/o rule)')
    ax.set_xticks(range(1, len(num_dfs) + 1))
    ax.set_xticklabels([str(num) for num in num_dfs.keys()])

    ax.autoscale(enable=False)
    ax.plot(ax.get_xlim(), (1, 1),
            **BACKGROUND_LINE_SETTINGS,
            linestyle='dashed')

Dropping 0 instances since they did not finish in all experiments
Dropped instances: set()


# How often and when are better upper bounds found?

In [30]:
df = get_df_hard_finished('default')


def bound_improvement_curve(row) -> List[Tuple[float, float]]:
    bound_gap = row.root_bounds__greedy_upper - row.opt
    if bound_gap == 0:
        return [(0, 1), (1, 1)]
    curve = [(0, 0)]
    for impr in row.upper_bound_improvements:
        rel_time = impr['runtime'] / row.runtimes__total
        rel_bound = 1 - (impr['new_bound'] - row.opt) / bound_gap
        curve.append((rel_time, rel_bound))
    return curve + [(1, 1)]


NUM_SAMPLES = 500
curves = [bound_improvement_curve(row) for row in df.itertuples()]
x = np.linspace(0, 1, NUM_SAMPLES)
y = np.linspace(0, 1, NUM_SAMPLES)

# TODO: find a smart numpy way to do this
z = np.zeros((NUM_SAMPLES, NUM_SAMPLES))
rel = 1 / len(curves)
for curve in curves:
    x_idx = 0
    for i in range(len(curve) - 1):
        x_left, curve_y = curve[i]
        x_right = curve[i + 1][0]
        y_idx = np.searchsorted(y, curve_y, side='right')
        while x_idx < NUM_SAMPLES and x[x_idx] <= x_right:
            z[x_idx][:y_idx] += rel
            x_idx += 1

with make_plot('bound-updates', (WIDTH_1COL, 0.85 * WIDTH_1COL)) as (fig, ax):
    contour = ax.contourf(x,
                          y,
                          z.transpose(),
                          cmap='Greys',
                          levels=np.linspace(0, 1, 11))

    # For some reason, the black polygon bugged out and leaves some white part
    # in the bottom right undrawn. This just lays a black background behind the
    # contour plot.
    ax.fill([0, 0, 1, 1], [0, 1, 1, 0], facecolor='black', zorder=-1)

    ax.set_aspect('equal')
    ax.set_xlabel('Runtime')
    ax.set_ylabel('Upper bound progress')

    formatter = plticker.PercentFormatter(xmax=1)
    ticks = np.linspace(0, 1, 6)
    ax.set_xticks(ticks)
    ax.tick_params('both', labelsize='x-small')
    ax.xaxis.set_major_formatter(formatter)
    ax.yaxis.set_major_formatter(formatter)

    cbar = fig.colorbar(contour,
                        ticks=ticks,
                        format=formatter,
                        fraction=0.046,
                        pad=0.04)
    cbar.ax.tick_params(labelsize='x-small')

    # Matplotlib by default cuts of the y-label, this mostly fixes the issue
    fig.subplots_adjust(left=-0.8)

## Is it worth doing upper bounds during the algo?

In [14]:
names = [
    'default', 'greedy-never', 'greedy-always-before-bounds',
    'greedy-always-before-expensive-reductions'
]
dfs = get_dfs_hard_finished(names)

base_df = dfs['greedy-never']
data = [
    dfs[name].runtimes__total / base_df.runtimes__total for name in [
        'default',
        'greedy-always-before-expensive-reductions',
        'greedy-always-before-bounds',
    ]
]
labels = [
    'Once,\nin the\nbeginning', 'Every loop,\nbefore expensive\nreductions',
    'Every loop,\nbefore\nbounds'
]

with make_plot('greedy-vs-off', (WIDTH_1COL, 2)) as (fig, ax):
    ax.boxplot(data, **BOXPLOT_SETTINGS)

    ax.set_xticklabels(labels)
    ax.tick_params(axis='both', which='major', labelsize='x-small')
    ax.yaxis.set_major_locator(plticker.MultipleLocator(base=0.5))

    # This is very weird. Re-enabling autoscale shouldn't be necessary
    # (and isn't in other boxplots). Anyway, this works (for now)...
    xlim = ax.get_xlim()
    ax.autoscale(enable=False)
    ax.plot(ax.get_xlim(), (1, 1), **BACKGROUND_LINE_SETTINGS, linestyle='dashed')
    ax.autoscale(enable=True)

Dropping 0 instances since they did not finish in all experiments
Dropped instances: set()


## Where do we put greedy?

In [83]:
names = [
    'default', 'greedy-always-before-bounds',
    'greedy-always-before-expensive-reductions'
]
dfs = get_dfs_hard_finished(names)

default_df = dfs['default']
always_early_df = dfs['greedy-always-before-bounds']
always_late_df = dfs['greedy-always-before-expensive-reductions']

x = always_late_df.runtimes__total / default_df.runtimes__total
y = always_early_df.runtimes__total / default_df.runtimes__total
mx = round(1.1 * max(x.max(), y.max()), ndigits=1)

with make_plot('where-greedy', (WIDTH_1COL, WIDTH_1COL)) as (fig, ax):
    ax.scatter(x, y, **SCATTER_SETTINGS)

    ax.set_xlabel('Every loop, before expensive reductions')
    ax.set_ylabel('Every loop, before bounds')
    ax.set_xlim((0, mx))
    ax.set_ylim((0, mx))

    for xs, ys in [((0, mx), (1, 1)), ((1, 1), (0, mx)), ((0, mx), (0, mx))]:
        ax.plot(xs, ys, **BACKGROUND_LINE_SETTINGS)

Dropping 0 instances since they did not finish in all experiments
Dropped instances: set()
